In [116]:
import gurobipy as gp
from gurobipy import GRB

# Create Parameters

# Default Paramters (Don't Change)

# Maximum discharge rate (MW)
maxDischargeRate = 2
# Maximum charge rate (MW)
maxChargeRate = -2
# Storage capacity of battery (MWh)
bStorage = 2
# Round Trip Efficiency of battery (unitless)
roundTripEfficiency = 0.81


# Changing Paramters 

# Storage capacity of battery at time zero (MWh)
bStorage0 = 1.5
# Price predictions for energy ($/MWh)
prices = (27.43703,
24.3,
29.77032,
28.60265,
24.58964,
27.96721,
27.82,
24.81493,
29.7,
22.77314,
28.26199,
27.82,
24.97438,
29.7,
20.53356,
28.02863,
27.82,
25.1368,
29.7,
21.33405,
100,
27.74008,
25.06542
)


try:

    # Create a new model
    m = gp.Model("batOptimser")

    # Create variables

    dispatchGen = {}
    dischargeBattery = {}
    chargeBattery = {}
    energyInStorage = {}
    dispatchCost = {}
    dispatchRevenue = {}
    dispatchProfit = {}
    
    for i, p in enumerate(prices):
        # Decision Variables
        dispatchGen[i] = m.addVar(name="dispatchGen")
        dischargeBattery[i] = m.addVar(lb=0,ub=maxDischargeRate,name="dischargeBattery")
        chargeBattery[i] = m.addVar(lb=maxChargeRate,ub=0,name="chargeBattery")
        energyInStorage[i] = m.addVar(lb=0,ub=bStorage,name="energyInStorage")
        
        dispatchCost[i] = -1*chargeBattery[i]*p/12
        dispatchRevenue[i] = dischargeBattery[i]*p/12
        dispatchProfit[i] = dispatchRevenue[i] - dispatchCost[i]
        
        # Constraints
        if (i > 0):
            preserveEnergyInStorage = m.addConstr(energyInStorage[i] == (energyInStorage[i-1]/12) - (roundTripEfficiency*chargeBattery[i-1]/12) - (dischargeBattery[i-1]/12))
        
        BatteryDispatchDefinition = m.addConstr(dispatchGen[i] == dischargeBattery[i] + chargeBattery[i]) 
        
    # More Constraints
    EnergyInStorageCapacity0 = m.addConstr(energyInStorage[0] == bStorage0)

    # Set objective
    m.setObjective(sum(dispatchProfit), GRB.MAXIMIZE)

    # Optimize model
    m.optimize()

    for v in m.getVars():
        print('%s %g' % (v.varName, v.x))

    print('\nMax profit for predicted prices: $%g' % m.objVal)
    
    if (dispatchGen[0].x > 0):
        print("Optimal solution this time interval: DISCHARGE")
    elif (dispatchGen[0].x < 0):
        print("Optimal solution this time interval: CHARGE")
    else:
        print("Optimal solution this time interval: DO NOTHING")
    

except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 46 rows, 92 columns and 158 nonzeros
Model fingerprint: 0x2838b041
Coefficient statistics:
  Matrix range     [8e-02, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [2e+00, 2e+03]
  RHS range        [2e+03, 2e+03]
Presolve removed 46 rows and 92 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.5300000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  2.530000000e+02
dispatchGen 0
dischargeBattery 2
chargeBattery -2
energyInStorage 2000
dispatchGen 0
dischargeBattery 0
chargeBattery 0
energyInStorage 166.667
dispatchGen 0
dischargeBattery 0
chargeBattery 0
energyInStorage 13.8889
dispatchGen 0
dischargeBattery 0
chargeBattery 0
energyInStorage 1.15741
dispatchGen 0
dischargeBattery 0
chargeBattery 0
energyInStorage 0.0964506
dispatchGen 0
dischargeBatt